### Цели исследования: оценка надежности заёмщиков, выявление влияния семейного положения, количества детей, уровня доходов и целей клиента на факт возврата кредита в срок. 

## Задание 1. Исследование

In [4]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
data = pd.read_csv('/datasets/data.csv')

Осмотр данных

In [8]:
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 11.4 MB


,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [52]:
data.sample()

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13003,0,-213.383683,34,Среднее,1,женат / замужем,0,M,сотрудник,0,135293.829632,на покупку автомобиля


In [53]:
data.tail()

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Вопросы вызвает столбец `days_employed`, где есть отрицательные и нереалистично большие значения.
Столбец `dob_days` объявлен, как возраст клиента в днях, хотя скорее всего там содержится количество полных лет.

Могу предположить, что отрицательные значения `days_employed` это трудовой стаж текущей работы в днях до подачи заявки на кредит. А положительные - ошибочные значения у пенсионеров и безработных. Там должен быть 0.

In [50]:
data[data['days_employed'] > 0].groupby('income_type').count()

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
пенсионер,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443


Смотрю пропущенные значения.

In [54]:
data.isnull().sum()

children               0
days_employed       2174
dob_days               0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружены пропущенные значения в столбцах `days_employed` — трудовой стаж в днях и `total_income` — доход в месяц. В столбце `total_income` пропуски заменены медианами, так как они приближены к типичным значениям выборки.
В столбце `days_employed` согласно описанной выше гипотезе у пенсионеров меняю пропущенные значения на 0, у остальных - на медиану по типу занятости. Положительные значения тоже меняю на ноль.

Проверяю в каких типах занятости есть пропущенные значения.


In [55]:
print(data[data['days_employed'].isnull()].groupby('income_type')['dob_days'].count())

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: dob_days, dtype: int64


У пенсионеров меняю пропущенные значения на 0.


In [56]:
indices = data['income_type'] == 'пенсионер'
data.loc[indices,'days_employed'] = data.loc[indices,'days_employed'].fillna(0)

У остальных меняю пропущенные значения на медиану по типу занятости

In [57]:
#Заполняем медианой пропущенные значения в выбранных типах занятости
def fillna_median(data, column, income_type, value_column):
    x = data[column] == income_type#индекс типа занятости
    m = data.loc[x, value_column].median()#медиана
    data.loc[x, value_column] = data.loc[x, value_column].fillna(m)
fillna_median(data, 'income_type', 'госслужащий', 'days_employed')
fillna_median(data, 'income_type', 'компаньон', 'days_employed')
fillna_median(data, 'income_type', 'предприниматель', 'days_employed')
fillna_median(data, 'income_type', 'сотрудник', 'days_employed')

Меняю на ноль все положительные значения в столбце

In [58]:
data.at[data['days_employed'] > 0, 'days_employed'] = 0

В столбце `total_income` заменяю пропуски медианами

In [59]:
median_income = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(median_income)

Делаю проверку

In [60]:
print(data[data['days_employed'].isnull()].groupby('income_type')['dob_days'].count())
data.isnull().sum()

Series([], Name: dob_days, dtype: int64)


children            0
days_employed       0
dob_days            0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Первоначально при осмотре данных было обнаружено, что значения в столбцах `days_employed` и `total_income` имеют тип `float`. 
В целях выполнения задания и для удобства дальнейшей работы с данными, тип был изменен на `int` с помощью метода `astype()`.

In [62]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверяю наличие дубликатов.

In [63]:
data.duplicated().sum()

54

Изучаю дубликаты.

In [64]:
data.loc[data.duplicated(keep=False), :]

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,-1574,46,среднее,1,женат / замужем,0,F,сотрудник,0,145017,высшее образование
520,0,-1574,35,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
541,0,-1574,57,среднее,1,женат / замужем,0,F,сотрудник,0,145017,сделка с подержанным автомобилем
554,0,-1574,60,среднее,1,женат / замужем,0,M,сотрудник,0,145017,покупка недвижимости
680,1,-2689,30,высшее,0,женат / замужем,0,F,госслужащий,0,145017,покупка жилья для семьи
1005,0,0,62,среднее,1,женат / замужем,0,F,пенсионер,0,145017,ремонт жилью
1191,0,0,61,среднее,1,женат / замужем,0,F,пенсионер,0,145017,операции с недвижимостью
1431,0,-1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
1511,0,0,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование
1681,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


Предоставленные данные не дают возможности точно различить заёмщиков, строки которых определяются как дубликаты. Есть высокая вероятность, что это разные люди с совпадающими характеристиками. Поэтому я не буду их удалять.

Далее провожу поиск дубликатов в столбцах со строковыми значениями.

Изучаю содержимое столбца `education`. Есть дубликаты из-за разных регистров. Возможно ошибки ввода.

In [65]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Перевожу все значения в нижний регистр и проверяю результат.

In [66]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Изучаю содержимое столбца `family_status`. Есть дубликаты из-за разных регистров. Возможно ошибки ввода.

In [67]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Перевожу все значения в нижний регистр и проверяю результат.

In [68]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Изучаю содержимое столбца `income_type`.

In [69]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Изучаю содержимое столбца `gender`. Есть один пропуск со значением `XNA`. Возможно ошибка ввода.

In [70]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [71]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


По другим данным не удалось определить пол. Поэтому меняют на случайное значение.

In [81]:
import random
data.at[10701, 'gender'] = random.choice(['F', 'M'])
data['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

Далее поиск ошибок и пропущенных значений в столбцах с числовыми значениями.
Изучаю содержимое столбца `dob_days`. Есть 101 пропуск с нулевым значением. Возможно ошибки ввода.

In [26]:
data['dob_days'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_days, dtype: int64

В столбце `dob_days` нулевые значения заменены медианами, так как они приближены к типичным значениям выборки.

In [82]:
median_dob_days = data[data['dob_days'] > 0]['dob_days'].median()
data.loc[data['dob_days'] == 0, 'dob_days'] = median_dob_days

Изучаю содержимое столбца `education_id`.

In [83]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Изучаю содержимое столбца `purpose`.

In [84]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Лемматизирую значения столбца `purpose`.

In [85]:
#Создаю список уникальных значений в столбце purpose
purposes_list = data['purpose'].unique()
lemmas = []
#Циклом лемматизирую содержание списка и добавляю лемметизированные значения в список lemmas
for phrase in purposes_list:
    lemmas.append(m.lemmatize(phrase))

Создаю словарь категорий по цели получения кредита. Цель получения кредита может влиять на надёжность заёмщика.

In [86]:
purposes_list

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [87]:
#Создаю словарь, где ключами являются категории целей получения кредита
purposes_dict = {
   'жилье': [],
   'автомобиль': [],
   'образование': [],
   'свадьба': [],
   'недвижимость': []
}
#Циклом нахожу соответствия между конкретной категорией и элементом в списке уникальных значений столбца purpose. 
for i in range(len(lemmas)):
       if 'жилье' in lemmas[i]:
           purposes_dict['жилье'].append(purposes_list[i])
       elif 'автомобиль' in lemmas[i]:
           purposes_dict['автомобиль'].append(purposes_list[i])
       elif 'образование' in lemmas[i]:
           purposes_dict['образование'].append(purposes_list[i])
       elif 'свадьба' in lemmas[i]:
           purposes_dict['свадьба'].append(purposes_list[i])
       elif 'недвижимость' in lemmas[i]:
           purposes_dict['недвижимость'].append(purposes_list[i])
#Заполняю значения словаря
print(purposes_dict)

{'жилье': ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'жилье', 'покупка своего жилья', 'покупка жилья для сдачи', 'ремонт жилью'], 'автомобиль': ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля'], 'образование': ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования', 'получение дополнительного образования', 'получение высшего образования', 'профильное образование', 'высшее образование', 'заняться высшим образованием'], 'свадьба': ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба'], 'недвижимость': ['покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимос

Создаю новый столбец с типами целей.

In [88]:
list = []
for j in data['purpose']:
    for key, value in purposes_dict.items():
        if j in value:
            list.append(key)
data['purpose_type'] = list

Проверяю результат.

In [89]:
data.head()

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Изучаю содержимое столбца `children`. Подозрительные значения -1 и 20.

In [90]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Проверяю есть ли взаимосвязь между подозрительными значениями и другими значениями. Взаимосвязь не выявлена.

In [91]:
data[data['children'] == 20]

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
606,20,-880,21.0,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья,жилье
720,20,-855,44.0,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости,недвижимость
1074,20,-3310,56.0,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования,образование
2510,20,-2714,59.0,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью,недвижимость
2941,20,-2161,43.0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля,автомобиль
3302,20,-2689,35.0,среднее,1,не женат / не замужем,4,F,госслужащий,0,145017,профильное образование,образование
3396,20,-1547,56.0,высшее,0,женат / замужем,0,F,компаньон,0,145017,высшее образование,образование
3671,20,-913,23.0,среднее,1,не женат / не замужем,4,F,сотрудник,0,101255,на покупку подержанного автомобиля,автомобиль
3697,20,-2907,40.0,среднее,1,гражданский брак,1,M,сотрудник,0,115380,на покупку подержанного автомобиля,автомобиль
3735,20,-805,26.0,высшее,0,не женат / не замужем,4,M,сотрудник,0,137200,ремонт жилью,жилье


Заменяют значения -1 и 20 на моду.

In [92]:
mode_children = data['children'].mode()
data.loc[data['children'] == -1, 'children'] = mode_children 
data.loc[data['children'] == 20, 'children'] = mode_children 

Создаю новый столбец с категориями заёмщиков в зависимости от количества детей.
Дети создают нагрузку на бюджет домохозяйства и могут негативно влиять на финансовую стабильность.

In [93]:
def children_type(x):
    if x == 0:
        return 'нет детей'
    if 0 < x < 3:
        return 'есть дети'
    if 3 < x <= 5:
        return 'многодетный / многодетная'
data['children_type'] = data['children'].apply(children_type)

Создаю новый столбец с категориями заёмщиков в зависимости от уровня дохода.
Возможно заёмщики с более высоким доходом имеют большую надёжность.

In [94]:
def total_income_type(x):
    if x > 150000:
        return 'очень высокий'
    if  80000 < x <= 150000:
        return 'высокий'
    if  50000 < x <= 80000:
        return 'средний'
    else:
        return 'низкий'
data['total_income_type'] = data['total_income'].apply(total_income_type)

## Задание 2. Описание выводов

### 1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [95]:
children_pivot = data.pivot_table(index=['children_type'], columns='debt', values='purpose', aggfunc='count')
children_pivot['ratio'] = (children_pivot[1] + children_pivot[0]) / children_pivot[1]
print(children_pivot, '\n')
print(round(children_pivot['ratio'][0],1),'% заёмщиков с детьми имели задолженность по кредиту', sep='')
print(round(children_pivot['ratio'][1],1),'% многодетных заёмщиков имели задолженность по кредиту', sep='')
print(round(children_pivot['ratio'][2],1),'% заёмщиков без детей имели задолженность по кредиту', sep='')

debt                           0     1      ratio
children_type                                    
есть дети                   6235   638  10.772727
многодетный / многодетная     46     4  12.500000
нет детей                  13086  1063  13.310442 

10.8% заёмщиков с детьми имели задолженность по кредиту
12.5% многодетных заёмщиков имели задолженность по кредиту
13.3% заёмщиков без детей имели задолженность по кредиту


Наблюдается незначительная зависимость между наличием детей и возвратом кредита в срок. 12.5% многодетных заёмщиков имели задолженность по кредиту. Среди многодетных заёмщиков таких было на 0.8% больше. А среди заёмщиков без детей - на 1.7% меньше.

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [96]:
family_pivot = data.pivot_table(index=['family_status'], columns='debt', values='purpose', aggfunc='count')
family_pivot['ratio'] = (family_pivot[1] + family_pivot[0]) / family_pivot[1]
print(family_pivot,'\n')
print(round(family_pivot['ratio'][0],1),'% заёмщиков в разводе имели задолженность по кредиту', sep='')
print(round(family_pivot['ratio'][1],1),'% заёмщиков среди вдовцов / вдов имели задолженность по кредиту', sep='')
print(round(family_pivot['ratio'][2],1),'% заёмщиков в гражданском браке имели задолженность по кредиту', sep='')
print(round(family_pivot['ratio'][3],1),'% женатых / замужних заёмщиков имели задолженность по кредиту', sep='')
print(round(family_pivot['ratio'][4],1),'% неженатых / незамужних заёмщиков имели задолженность по кредиту', sep='')

debt                       0    1      ratio
family_status                               
в разводе               1110   85  14.058824
вдовец / вдова           897   63  15.238095
гражданский брак        3789  388  10.765464
женат / замужем        11449  931  13.297530
не женат / не замужем   2539  274  10.266423 

14.1% заёмщиков в разводе имели задолженность по кредиту
15.2% заёмщиков среди вдовцов / вдов имели задолженность по кредиту
10.8% заёмщиков в гражданском браке имели задолженность по кредиту
13.3% женатых / замужних заёмщиков имели задолженность по кредиту
10.3% неженатых / незамужних заёмщиков имели задолженность по кредиту


Наблюдается незначительная зависимость между семейным положением и возвратом кредита в срок.
Заёмщики среди вдовцов и вдов, а также находящихся в разводе чуть чаще имеют задолженность - 15.2% и 14.1% соотвественно. Напротив, заёмщики в гражданском браке и неженатые / незамужние более ответственны - около 10.5%. Среди женатых / замужних заёмщиков 13.3% имели задолженность по кредиту.

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [97]:
income_pivot = data.pivot_table(index=['total_income_type'], columns='debt', values='purpose', aggfunc='count')
income_pivot['ratio'] = (income_pivot[1] + income_pivot[0]) / income_pivot[1]
print(income_pivot, '\n')
print(round(income_pivot['ratio'][2],1),'% заёмщиков с очень высоким доходом имели задолженность по кредиту', sep='')
print(round(income_pivot['ratio'][0],1),'% заёмщиков с высоким доходом имели задолженность по кредиту', sep='')
print(round(income_pivot['ratio'][3],1),'% заёмщиков со средним доходом имели задолженность по кредиту', sep='')
print(round(income_pivot['ratio'][1],1),'% заёмщиков с низким доходом имели задолженность по кредиту', sep='')

debt                  0    1      ratio
total_income_type                      
высокий            9224  841  11.967895
низкий              349   23  16.173913
очень высокий      8458  726  12.650138
средний            1753  151  12.609272 

12.7% заёмщиков с очень высоким доходом имели задолженность по кредиту
12.0% заёмщиков с высоким доходом имели задолженность по кредиту
12.6% заёмщиков со средним доходом имели задолженность по кредиту
16.2% заёмщиков с низким доходом имели задолженность по кредиту


Наблюдается незначительная зависимость между уровнем дохода и возвратом кредита в срок.
Заёмщики с низким доходом чаще других имеют задолженность(16.2%). Остальные категории - около 12%.

### 4. Как разные цели кредита влияют на его возврат в срок?

In [98]:
purpose_pivot = data.pivot_table(index=['purpose_type'], columns='debt', values='purpose', aggfunc='count')
purpose_pivot['ratio'] = (purpose_pivot[1] + purpose_pivot[0]) / purpose_pivot[1]
print(purpose_pivot, '\n')
print(round(purpose_pivot['ratio'][0],1),'% заёмщиков целью которых был автомобиль имели задолженность по кредиту', sep='')
print(round(purpose_pivot['ratio'][1],1),'% заёмщиков целью которых было жилье имели задолженность по кредиту', sep='')
print(round(purpose_pivot['ratio'][2],1),'% заёмщиков целью которых было образование имели задолженность по кредиту', sep='')
print(round(purpose_pivot['ratio'][3],1),'% заёмщиков целью которых была недвижимость имели задолженность по кредиту', sep='')
print(round(purpose_pivot['ratio'][4],1),'% заёмщиков целью которых была свадьба имели задолженность по кредиту', sep='')

debt             0    1      ratio
purpose_type                      
автомобиль    3912  403  10.707196
жилье         4165  308  14.522727
недвижимость  5893  474  13.432489
образование   3652  370  10.870270
свадьба       2162  186  12.623656 

10.7% заёмщиков целью которых был автомобиль имели задолженность по кредиту
14.5% заёмщиков целью которых было жилье имели задолженность по кредиту
13.4% заёмщиков целью которых было образование имели задолженность по кредиту
10.9% заёмщиков целью которых была недвижимость имели задолженность по кредиту
12.6% заёмщиков целью которых была свадьба имели задолженность по кредиту


Наблюдается незначительная зависимость между целями кредита и возвратом кредита в срок.
Около 11% заёмщиков, приобретающих недвижимость и автомобиль имеют задолженность. Чуть более дисциплинированны клиенты, целью которых была свадьба(12.6%). Самый большой процент среди желающих оплатить обучение и приобрести жильё - 13.4% и 14.5% соответственно.  

### Общий вывод:
- Представленные данные дают возможность сделать вывод об отсутствии значительной зависимости между проанализированными характеристиками заёмщика и возвратом кредита в срок. Разница значений по каждой категории в пределах 3-5%.</big>
- В ходе предобработки данных отсутствующие значения о доходах 10% заемщиков были заменены медианой. Менее 0.5% значений о количестве детей признаны ошибочными и заменены на самое частотное значение. Это может оказывать незначительное воздействие на финальные выводы.
- Также в значениях об образовании и семейном положении были исправлены дубликаты, предположительно возникшие из-за ошибок ввода.